In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install tflearn 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 5.2 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=9f64dd98211f51d2da4ade1110b26a04fec996caa381e25aea8f593965ff69be
  Stored in directory: /root/.cache/pip/wheels/65/9b/15/cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import random
img_size=224
training_data=[]
#!unzip drive/My\ Drive/NN\ Project/NN\ Dataset.zip
for dirname, _, filenames in os.walk('Train'):
    for filename in filenames:
    #print(os.path.join(filename))
    #data=cv2.imread()
      path=os.path.join(dirname, filename)
        
      label=filename.replace('_','.')
      label=label.split('.')
      label=label[0]
      img=cv2.imread(path,cv2.IMREAD_COLOR)
      img = img[:, :, [2, 1, 0]]
      img=cv2.resize(img,(img_size,img_size))
      imgfliplr=cv2.flip(img,1)
      imgrotate = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
      imgflipud=cv2.flip(img,0)
      if(label=="Basketball"):
          label=[1,0,0,0,0,0]
      elif(label=="Football"):
          label=[0,1,0,0,0,0]
      elif(label=="Rowing"):
          label=[0,0,1,0,0,0]
      elif(label=="Swimming"):
          label=[0,0,0,1,0,0] 
      elif(label=="Tennis"):
          label=[0,0,0,0,1,0]
      elif(label=="Yoga"):
          label=[0,0,0,0,0,1]
            
      training_data.append([np.array(img),label])
      training_data.append([np.array(imgfliplr),label])
      training_data.append([np.array(imgrotate),label])
      training_data.append([np.array(imgflipud),label])
      
        
random.shuffle(training_data)
print(len(training_data))

    

6724


In [10]:
x_train=np.array([i[0]for i in training_data]).reshape(-1,img_size,img_size,3)


print(x_train.shape)
#print(x_train)
y_train=[i[1]for i in training_data]
#print(y_train)

    
      


    
    

(6724, 224, 224, 3)


In [11]:
LR=0.001
tf.compat.v1.reset_default_graph()
conv_input=input_data(shape=[None,img_size,img_size,3],name="input")
conv1 = conv_2d(conv_input, 32, 3, activation='relu')
pool1 = max_pool_2d(conv1, 5)
conv2 = conv_2d(pool1, 64, 3, activation='relu')
pool2 = max_pool_2d(conv2, 5)


conv3 = conv_2d(pool2, 128, 3, activation='relu')
pool3 = max_pool_2d(conv3, 5)

conv4 = conv_2d(pool3, 256,3,activation='relu')
pool4 = max_pool_2d(conv4,5)


conv5 = conv_2d(pool4, 128,3,activation='relu')
pool5 = max_pool_2d(conv5,5)


conv6 = conv_2d(pool5, 64,3,activation='relu')
pool6 = max_pool_2d(conv6,5)

conv7 = conv_2d(pool6, 32,3,activation='relu')
pool7 = max_pool_2d(conv7,5)


# conv3 = conv_2d(pool2, 256, 5, activation='relu')
# pool3 = max_pool_2d(conv3, 5)

# conv4 = conv_2d(pool3, 64, 5, activation='relu')
# pool4 = max_pool_2d(conv4, 5)

# conv5 = conv_2d(pool4, 32, 5, activation='relu')
# pool5 = max_pool_2d(conv5, 5)

fully_layer = fully_connected(pool7, 1000, activation='relu')
fully_layer = dropout(fully_layer, 0.5)

cnn_layers = fully_connected(fully_layer, 6, activation='softmax')

cnn_layers = regression(cnn_layers, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
model = tflearn.DNN(cnn_layers, tensorboard_dir='log', tensorboard_verbose=3)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
MODEL_NAME="Sports_Classification"
model.fit({'input': x_train}, {'targets': y_train}, n_epoch=10,
          validation_set=(0.2),
          snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 849  | total loss: 0.28556 | time: 429.837s
| Adam | epoch: 010 | loss: 0.28556 - acc: 0.9052 -- iter: 5376/5379
Training Step: 850  | total loss: 0.26741 | time: 436.096s
| Adam | epoch: 010 | loss: 0.26741 - acc: 0.9116 | val_loss: 0.46637 - val_acc: 0.8558 -- iter: 5379/5379
--


In [13]:
model.save('model.tfl')

In [15]:
from google.colab import files
files.download('model.tfl.data-00000-of-00001')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
import matplotlib.pyplot as plt
output=pd.DataFrame(columns=['image_name','label'])
i=0
for dirname, _, filenames in os.walk('Test'):
    for filename in filenames:

        
    #print(os.path.join(filename))
    #data=cv2.imread()
      path=os.path.join(dirname, filename)
        
#         label=filename.replace('_','.')
#         label=label.split('.')
#         label=label[0]
      img=cv2.imread(path,cv2.IMREAD_COLOR)
      img = img[:, :, [2, 1, 0]]
      test_img=cv2.resize(img,(img_size,img_size))
#          if(label=="Basketball"):
#             label=[1,0,0,0,0,0]
#         elif(label=="Football"):
#             label=[0,1,0,0,0,0]
#         elif(label=="Rowing"):
#             label=[0,0,1,0,0,0]
#         elif(label=="Swimming"):
#             label=[0,0,0,1,0,0] 
#         elif(label=="Tennis"):
#             label=[0,0,0,0,1,0]
#         elif(label=="Yoga"):
#             label=[0,0,0,0,0,1]
      test_img=test_img.reshape(img_size,img_size,3)
      prediction=model.predict([test_img])[0]
      max_index = np.argmax(prediction)
      new_row={'image_name':filename,'label':max_index}
      output=output.append(new_row,ignore_index=True)
      # fig = plt.figure(figsize=(10, 10))
      # ax = fig.add_subplot(111)
      # ax.imshow(img)
      # print(f"Basketball: {prediction[0]}, Football: {prediction[1]},Rowing: {prediction[2]},Swimming: {prediction[3]},Tennis: {prediction[4]},Yoga: {prediction[5]}")
        
      # plt.show()
      # if(i==40):
      #     break
      # i+=1
print(output)
output.to_csv('file1.csv',index=False)

    image_name label
0      644.jpg     5
1      682.jpg     1
2      570.jpg     5
3       90.jpg     3
4      353.jpg     2
..         ...   ...
683    247.jpg     1
684    324.jpg     3
685    325.jpg     3
686    427.jpg     4
687    304.jpg     1

[688 rows x 2 columns]
